In [1]:
#!pip3 install langchain

In [16]:
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import ElasticVectorSearch
from langchain.text_splitter import CharacterTextSplitter
from urllib.request import urlopen
import os, json
 
load_dotenv()
 
openai_api_key=os.getenv('OPENAI_API_KEY')
elastic_user=os.getenv('ES_USER')
elastic_password=os.getenv('ES_PASSWORD')
elastic_endpoint=os.getenv("ES_ENDPOINT")
elastic_index_name='elastic-vector-search'

In [17]:
import json
 
 
# Load data into a JSON object
with open('workplace-docs.json') as f:
   workplace_docs = json.load(f)
 
print(f"Successfully loaded {len(workplace_docs)} documents")

metadata = []
content = []
 
for doc in workplace_docs:
  content.append(doc["content"])
  metadata.append({
      "name": doc["name"],
      "summary": doc["summary"],
      "rolePermissions":doc["rolePermissions"]
  })
 
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
docs = text_splitter.create_documents(content, metadatas=metadata)

Created a chunk of size 245, which is longer than the specified 100
Created a chunk of size 288, which is longer than the specified 100
Created a chunk of size 204, which is longer than the specified 100
Created a chunk of size 281, which is longer than the specified 100
Created a chunk of size 249, which is longer than the specified 100
Created a chunk of size 285, which is longer than the specified 100
Created a chunk of size 298, which is longer than the specified 100
Created a chunk of size 270, which is longer than the specified 100
Created a chunk of size 224, which is longer than the specified 100
Created a chunk of size 288, which is longer than the specified 100
Created a chunk of size 260, which is longer than the specified 100
Created a chunk of size 199, which is longer than the specified 100
Created a chunk of size 290, which is longer than the specified 100
Created a chunk of size 251, which is longer than the specified 100
Created a chunk of size 195, which is longer tha

Successfully loaded 15 documents


In [18]:
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

url = f"https://{elastic_user}:{elastic_password}@{elastic_endpoint}:9200"

ssl_verify = {
    "verify_certs": True,
    "basic_auth": (elastic_user, elastic_password),
    "ca_certs": "./http_ca.crt"
}

es = ElasticVectorSearch.from_documents( 
                            docs,
                            embedding = embeddings, 
                            elasticsearch_url = url, 
                            index_name = elastic_index_name, 
                            ssl_verify = ssl_verify)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/langchain/vectorstores/elastic_vector_search.py:143: UserWarning: ElasticVectorSearch will be removed in a future release. SeeElasticsearch integration docs on how to upgrade.
  warnings.warn(


In [19]:
es

In [20]:
def showResults(output):
  print("Total results: ", len(output))
  for index in range(len(output)):
    print(output[index])

In [21]:
query = "work from home policy"
result = es.similarity_search(query=query)

showResults(result)

Total results:  4
page_content='The purpose of this full-time work-from-home policy is to provide guidelines and support for employees to conduct their work remotely, ensuring the continuity and productivity of business operations during the COVID-19 pandemic and beyond.\nScope' metadata={'name': 'Work From Home Policy', 'summary': 'This policy outlines the guidelines for full-time remote work, including eligibility, equipment and resources, workspace requirements, communication expectations, performance expectations, time tracking and overtime, confidentiality and data security, health and well-being, and policy reviews and updates. Employees are encouraged to direct any questions or concerns', 'rolePermissions': ['demo', 'manager']}
page_content='This work-from-home policy will be reviewed periodically and updated as necessary, taking into account changes in public health guidance, business needs, and employee feedback.\nQuestions and Concerns' metadata={'name': 'Work From Home Polic